Source : https://github.com/A-bone1/FSNS-tfrecord-generate/blob/master/generate_tfrecord_PNG.py

In [107]:
from random import shuffle
import numpy as np
import glob
import tensorflow
from tensorflow.compat import v1 as tf
import cv2
import sys
import os
import PIL.Image as Image

In [99]:
def encode_utf8_string(text, length, null_char_id=0):
    char_ids_padded = [null_char_id]*length
    char_ids_unpadded = [null_char_id]*len(text)
    for i in range(len(text)):
        hash_id = ord(text[i])
        char_ids_padded[i] = hash_id
        char_ids_unpadded[i] = hash_id
    return char_ids_padded, char_ids_unpadded

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

In [83]:
word_bag = {}
with open('word_bag.txt', encoding="utf-8") as dict_file:
    for line in dict_file:
        (key, value) = line.strip().split('\t')
        word_bag[value] = int(key)
print(len(word_bag))

1395


In [100]:
image_path = 'data/dataset_1/images/*.png'
addrs_image = glob.glob(image_path)

label_path = 'data/dataset_1/labels/*.txt'
addrs_label = glob.glob(label_path)

print(len(addrs_image))
print(len(addrs_label))

29
29


In [65]:
for text in open(addrs_label[j], encoding="utf-8"):
    #print(text.encode('unicode_escape'))
    print(repr(text.replace(" ", "\t")).replace("'", ""))

장비분류\t:\t눈장식\n
STR\t:\t+24\t(6\t+18)\n
DEX\t:\t+33\t(6\t+27)\n
INT\t:\t+24\t(6\t+18)\n
LUK\t:\t+33\t(6\t+27)\n
공격력\t:\t+1\n
마력\t:\t+1\n
방어력\t:\t+100\n
업그레이드\t가능\t횟수:\t3\t(복구\t가능\t횟수:\t0)


In [98]:
for text in open(addrs_label[j], encoding="utf-8"):
    for t in text:
        print(t, ord(t))
    break

장 51109
비 48708
분 48516
류 47448
  32
: 58
  32
눈 45576
장 51109
식 49885

 10


In [113]:
tfrecord_writer  = tf.python_io.TFRecordWriter("tfexample_train")

#for j in range(0, int(len(addrs_image))): 
for j in range(0, 1): 
    print('Train data: {}/{}'.format(j,int(len(addrs_image))))
    #sys.stdout.flush()

    img = Image.open(addrs_image[j])

    #img = img.resize((600, 150), Image.ANTIALIAS)
    np_data = np.array(img)
    image = tf.image.convert_image_dtype(np_data, dtype=tf.uint8)
    encode_image = tf.image.encode_png(image).numpy()

    '''
    with tf.Session() as sess:
        image_data = sess.run(tf.image.encode_png(image))
        sess.close()
    '''
    for text in open(addrs_label[j], encoding="utf-8"):
        char_ids_padded, char_ids_unpadded = encode_utf8_string(
            text=text,
            length=400)


    example = tf.train.Example(features=tf.train.Features(
            feature={
                'image/encoded': _bytes_feature(encode_image),
                'image/format': _bytes_feature(b"PNG"),
                'image/width': _int64_feature([np_data.shape[1]]),
                'image/orig_width': _int64_feature([np_data.shape[1]]),
                'image/class': _int64_feature(char_ids_padded),
                'image/unpadded_class': _int64_feature(char_ids_unpadded),
                'image/text': _bytes_feature(bytes(text, 'utf-8')),
                # 'height': _int64_feature([crop_data.shape[0]]),
            }
        ))
    tfrecord_writer.write(example.SerializeToString())

#tfrecord_writer.close()
#sys.stdout.flush()

Train data: 0/29


In [115]:
reader = tf.TFRecordReader()
_, ex = reader.read("tfexample_train")

RuntimeError: Readers are not supported when eager execution is enabled. Instead, please use tf.data to get data into your model.

In [116]:
with tf.Session() as sess:
    reader = tf.TFRecordReader()
    _, ex = reader.read("tfexample_train")

AttributeError: &#39;str&#39; object has no attribute &#39;queue_ref&#39;